In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import pandas as pd


In [2]:
from bs4 import BeautifulSoup

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
import pymorphy2 # Морфологический анализатор.
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, rand

In [4]:
# отключаем warnings
import warnings

warnings.filterwarnings('ignore')

In [5]:
# cleantext = BeautifulSoup(raw_html, "lxml").text

In [6]:
df_train_raw = pd.read_csv('data/train.csv', sep='\t')

In [7]:
df_train_raw.head()

,id,name,description,target
0,0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [8]:
df_train_raw.shape

(200000, 4)

In [9]:
df_test = pd.read_csv('data/test.csv', sep='\t')

In [10]:
df_test.head()

,id,name,description
0,200000,Дизайнер-консультант мебели,<p><strong>Обязанности:</strong></p> <ul> <li>...
1,200001,Продавец-консультант (ТЦ на Пушкина),<p><strong>Обязанности</strong>:</p> <p>∙ конс...
2,200002,Менеджер по продажам,<p>Торговый Дом «Форт» это ведущая компания Пе...
3,200003,Продавец-консультант в магазин одежды (ТЦ Волн...,<p><strong>Требуются продавцы консультанты в м...
4,200004,Специалист по охране труда,<strong>Обязанности:</strong> <ul> <li> <p>осу...


In [11]:
df_test.shape

(170179, 3)

In [12]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
    

In [13]:
def prepare_data(dataframe):
    df = dataframe.copy()
    # clearhtml
    df['description'] = df['description'].apply(lambda x : cleanhtml(x) )
    # replace &quot to "
    df['description'] = df['description'].str.replace('&quot;','"')
    df['description'] = df['description'].str.replace('\u200b','')
    # uni name and description
    df['text'] = df['name'] + ' ' + df['description'] 
    # drop
    df = df.drop(columns=['name', 'description', 'id'], axis=1)
    return df
    

In [14]:
def f_tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()
    if type(s) == str:
        t = s.split(' ')
    else:
        t = s
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):
                f.append(wrd.normal_form)
    return f

In [15]:
N = df_train_raw.shape[0]

In [16]:
%%time 
df_train = prepare_data(df_train_raw[:N])

CPU times: user 9.53 s, sys: 860 ms, total: 10.4 s
Wall time: 10.4 s


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
target    200000 non-null int64
text      200000 non-null object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [18]:
df_tmp_test = prepare_data(df_test[:N])


In [19]:
df_tmp_test.head()

,text
0,Дизайнер-консультант мебели Обязанности: Рабо...
1,Продавец-консультант (ТЦ на Пушкина) Обязаннос...
2,Менеджер по продажам Торговый Дом «Форт» это в...
3,Продавец-консультант в магазин одежды (ТЦ Волн...
4,Специалист по охране труда Обязанности: осущ...


In [20]:
df_id = df_test[:N].drop(columns=['name', 'description'], axis=1)
# df_id = df_id.index

In [21]:
df_id.index

RangeIndex(start=0, stop=170179, step=1)

In [22]:
label = 'target'
RND_SEED = 123
idx_features = df_train.columns != label

In [23]:
X = df_train.loc[:, idx_features].values.tolist()
y = df_train.loc[:, ~idx_features].values
tests = df_tmp_test["text"].values

In [24]:
texts = []
for item in X:
    texts.append(item[0])

In [25]:
texts[1000]

'Технический специалист О компании: SUPPORT.UA - Мультикатегорийный сервис-провайдер № 1 в Украине. SUPPORT.UA - украинский дистрибьютор сервисов по установке, настройке и обслуживанию техники для розничных потребителей. Наша команда - это высококвалифицированные специалисты, которые предоставляют бесперебойный сервис высочайшего уровня. Support.ua ищет Технического Эксперта, специалиста по настройке компьютерной техники и мобильных устройств. Качества, которыми должен обладать наш сотрудник:  Желание работать в сфере настройки и обслуживания техники, развиваться и повышать свой уровень знаний на практике Навыки настройки цифровых устройств, знание Android, iOS, Windows Умение ориентироваться в различных категориях товара и способах оптимизации их работы Обучаемость, активность, коммуникабельность, нацеленность на результат  Мы предоставляем:  Официальное оформление с первого рабочего дня Интересную работу в динамично развивающейся компании Возможности для самореализации, профессиональ

In [27]:
def run_trials_template(X, y, params,evals=100, scoring='roc_auc', n_splits = 5, RND_SEED=123):
    
    def hyperopt_cv(X, y, params):
        X_ = X.copy()
        # Отделяем параметры лог регрессии в отдельный словарь
        lm_params = {}
        tfidf_params = {}
        blackbox = None
        for key, value in params.items():
            if key.startswith('glob'):
                continue                
            elif key.startswith('lm'):
                lm_params[key.split('_', 1)[1]] = value
            elif key.startswith('tfidf'):
                tfidf_params[key.split('_', 1)[1]] = value
            elif key.startswith('blackbox_model'): # FIXME
                blackbox = value

        if blackbox is None:
            print("can not find a blackbox model! Stop running")
            return -1

        model = Pipeline([
                ('union', FeatureUnion(
                        # Use FeatureUnion to combine the features from subject and body
                    transformer_list=[
                            # Pipeline for standard bag-of-words model
                        ('vec', Pipeline([
                                ('tfidf', TfidfVectorizer(**tfidf_params)),
                            ])
                        ),
                    ],# end  transformer_list
                        # weight components in FeatureUnion
                    transformer_weights={
                        'vec': 0.8,
                    }) # end FeatureUnion
                ),# end union
                ('clf', blackbox(**lm_params)),
        ]) # end pipeline

        cv = StratifiedKFold(
                n_splits=n_splits, shuffle=True, 
                random_state=RND_SEED)

        scores = cross_val_score(
            model, X_, y,
            scoring='roc_auc', 
            cv=cv, 
        n_jobs=-1)

        return scores.mean(), scores.std()
    
    def f(params):
        acc, std = hyperopt_cv(X, y, params)
        return {'loss': -acc, 'qscore': -acc, 'qscore_std': std, 'status': STATUS_OK}

    trials = Trials()
    best = fmin(
        f, 
        params, 
        algo=tpe.suggest, 
        max_evals=evals, 
        trials=trials, 
        verbose=10)
    
    return trials

In [28]:
%%time 
# Схема кросс-валидации

RND_SEED = 123
space4_lm_mnb = {
    'glob': None,
    'lm_alpha': 1.5,
    'tfidf_norm': None, 
    'tfidf_smooth_idf' :False,
    'blackbox_model':MultinomialNB
}
       

space4_lm_lg = {
    'glob': None,
    'lm_penalty': hp.choice('penalty', ['l2']),
    'lm_C': hp.loguniform('C', -1, 3),
    'lm_class_weight': hp.choice('class_weight', [None, 'balanced']),
    'lm_random_state': RND_SEED,
#     'tfidf_tokenizer' : f_tokenizer,
    'tfidf_norm': None, 
    'tfidf_smooth_idf' :False,
    'blackbox_model': LogisticRegression
}  

trials = run_trials_template(texts, y, space4_lm_lg)

Process ForkPoolWorker-445:
Process ForkPoolWorker-444:
Process ForkPoolWorker-448:
Process ForkPoolWorker-442:
Process ForkPoolWorker-447:
Process ForkPoolWorker-443:
Traceback (most recent call last):
Process ForkPoolWorker-446:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _boo

KeyboardInterrupt: 

In [ ]:
trials

In [ ]:
def trials_df(trials):
    '''
    Функция форматирует результаты hyperopt в dataframe
    '''
    tr_dict = []
    for t in trials:
        trial = dict()
        for k, v in t['misc']['vals'].items():
            trial[k] = v[0]

        trial['qscore'] = -t['result']['qscore']
        trial['qscore_std'] = -t['result']['qscore_std']
        tr_dict.append(trial)

    df_res = pd.DataFrame.from_dict(tr_dict)
    df_res = df_res.sort_values('qscore', ascending=False)
    
    return df_res

In [ ]:
df_trials = trials_df(trials)

In [ ]:
df_trials.head()

In [33]:
model.fit(texts, y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_...former_weights={'text': 0.5})), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [34]:
predictions = model.predict(tests)

In [35]:
df_id['target'] = pd.DataFrame(predictions.tolist())

In [48]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d_%H-%M')
df_id.to_csv(path_or_buf='./submission/submission_{today}.cvs'.format(today=today), index=False, sep=',')

'2018-03-09_09:16'